In [1]:
import time
import pandas as pd
import pandapower as pp
import numpy as np
import matplotlib.pyplot as plt
import os
from pandapower.plotting.plotly import pf_res_plotly
from pandapower.plotting.plotly import simple_plotly

import Utils.Network as Network
import Utils.Limits as Limits
import Utils.Market as Market
import Utils.utils as utils

# Import Network

In [ ]:
from pickletools import markobject


input_path = '../Data/Input'
output_path = '../Data/Output'

network_obj = Network.Network(input_path)

feeder_colors = ['#ff7f0e', '#9467bd']
feeder_colors_after = [utils.scale_lightness(c, 0.6) for c in feeder_colors]

meaningful_days = [15, 83, 162, 241, 324]
timesteps = network_obj.get_meaningful_days_timesteps(meaningful_days)

limits_obj = Limits.Limits(network_obj)
limits_obj.load_limits("solution_limits.npy")
market_obj = Market.Market(network_obj, limits_obj)

In [ ]:
ga_obj = limits_obj.calculate_limits()

In [ ]:
plot_P(network_obj, 4, limits=limits_obj.limits)

In [3]:
network_obj.load_time_series_at_timestep(network_obj.net, 1)

In [ ]:
market_obj.submit_orders()

In [ ]:
market_obj.market_clearing()

In [ ]:
# empty network
net = pp.create_empty_network()

# HV bus (20 kV) and LV (0,4 kV)
bus1 = pp.create_bus(net, vn_kv=20, name="HV Bus")  # Bus source 20 kV
bus2 = pp.create_bus(net, vn_kv=20, name="HT Bus 2")  # Bus avant transfo
bus3 = pp.create_bus(net, vn_kv=0.4, name="LT Bus")  # main LV bus

# prosumers busses
bus4 = pp.create_bus(net, vn_kv=0.4, name="bus C1")
bus5 = pp.create_bus(net, vn_kv=0.4, name="bus C2")
bus6 = pp.create_bus(net, vn_kv=0.4, name="bus C3")
bus7 = pp.create_bus(net, vn_kv=0.4, name="bus C4")
bus8 = pp.create_bus(net, vn_kv=0.4, name="bus C5")
bus9 = pp.create_bus(net, vn_kv=0.4, name="bus C6")
bus10 = pp.create_bus(net, vn_kv=0.4, name="bus C7")
bus11 = pp.create_bus(net, vn_kv=0.4, name="bus C8")

# HV line between source and transfo
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1, std_type="NA2XS2Y 1x185 RM/25 12/20 kV", name="Ligne HV")

#pp.create_shunt(net, bus=bus3, p_mw=0, q_mvar=-2, name="Régulateur de Tension")


# Transformateur 20kV/0,4kV
pp.create_transformer_from_parameters(
    net,
    hv_bus=bus2, lv_bus=bus3,
    sn_mva=2.5,        # Puissance nominale du transfo (1.25 MVA)
    vn_hv_kv=20,        # Tension nominale HT (20 kV)
    vn_lv_kv=0.4,       # Tension nominale BT (0.4 kV)
    vk_percent=6,       # Tension de court-circuit en %
    vkr_percent=0.5,    # Pertes résistives (%)
    pfe_kw=1.5,         # Pertes à vide (kW)
    i0_percent=0.1,     # Courant à vide (%)
    shift_degree=0,     # Décalage angulaire (Dyn5)
    name="Transfo 2.5 MVA"
)

# prosumers lines
pp.create_line(net, from_bus=bus3, to_bus=bus4, length_km=0.1, std_type="NAYY 4x150 SE", name="line LV 1")
pp.create_line(net, from_bus=bus3, to_bus=bus5, length_km=0.3, std_type="NAYY 4x150 SE", name="line LV 2")
pp.create_line(net, from_bus=bus3, to_bus=bus6, length_km=0.4, std_type="NAYY 4x150 SE", name="line LV 3")
pp.create_line(net, from_bus=bus3, to_bus=bus7, length_km=0.5, std_type="NAYY 4x150 SE", name="line LV 4")
pp.create_line(net, from_bus=bus3, to_bus=bus8, length_km=0.5, std_type="NAYY 4x150 SE", name="line LV 5")
pp.create_line(net, from_bus=bus3, to_bus=bus9, length_km=0.5, std_type="NAYY 4x150 SE", name="line LV 6")
pp.create_line(net, from_bus=bus3, to_bus=bus10, length_km=0.5, std_type="NAYY 4x150 SE", name="line LV 7")
pp.create_line(net, from_bus=bus3, to_bus=bus11, length_km=0.5, std_type="NAYY 4x150 SE", name="line LV 8")

# source 20 kV
pp.create_ext_grid(net, bus=bus1, vm_pu=1.02, name="Bus source")

# Prosumers consumption
pp.create_load(net, bus=bus4, p_mw=0.008, q_mvar=0.0003, name="Charge C1")
pp.create_load(net, bus=bus5, p_mw=0.005, q_mvar=0.0004, name="Charge C2")
pp.create_load(net, bus=bus6, p_mw=0.007, q_mvar=0.00035, name="Charge C3")
pp.create_load(net, bus=bus7, p_mw=0.006, q_mvar=0.00025, name="Charge C4")
pp.create_load(net, bus=bus8, p_mw=0.008, q_mvar=0.0004, name="Charge C5")
pp.create_load(net, bus=bus9, p_mw=0.007, q_mvar=0.00025, name="Charge C6")
pp.create_load(net, bus=bus10, p_mw=0.004, q_mvar=0.00025, name="Charge C7")
pp.create_load(net, bus=bus11, p_mw=0.003, q_mvar=0.00025, name="Charge C8")

# prosumers prodcution
pp.create_sgen(net, bus=bus4, p_mw=0.005, name="PV C1")
pp.create_sgen(net, bus=bus5, p_mw=0.007, name="PV C2")
pp.create_sgen(net, bus=bus6, p_mw=0.002, name="PV C3")
pp.create_sgen(net, bus=bus7, p_mw=0.003, name="PV C4")
pp.create_sgen(net, bus=bus8, p_mw=0.004, name="PV C5")
pp.create_sgen(net, bus=bus9, p_mw=0.003, name="PV C6")
pp.create_sgen(net, bus=bus10, p_mw=0.005, name="PV C7")
pp.create_sgen(net, bus=bus11, p_mw=0.003, name="PV C8")

v = .05
net.load.p_mw = -v
net.load.q_mvar = 0
net.sgen.p_mw = 0
net.sgen.q_mvar = 0
net.load.p_mw.plot()
net.sgen.p_mw.plot()

pp.runpp(net)
_ = pf_res_plotly(net)